# Redes Perceptron de múltiplas camadas - não binário

* Rede feed-forward
* Treinamento Supervisionado
* Algoritmo Backpropagation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
# importando dataset
dataset = pd.read_csv("frutas.csv")
dataset.head()

,ph,peso(kg),diametro(cm),fruta
0,2.980056,0.059878,8.303492,0
1,2.619839,0.059534,9.563558,0
2,2.983526,0.040029,8.921555,0
3,2.827375,0.046539,5.919455,0
4,2.979685,0.055912,9.392527,0


In [3]:
# extraindo características (x)
x = np.array(dataset.loc[:, ["ph", "peso(kg)", "diametro(cm)"]])
x

array([[ 2.98005606e+00,  5.98781498e-02,  8.30349159e+00],
       [ 2.61983873e+00,  5.95344254e-02,  9.56355780e+00],
       [ 2.98352600e+00,  4.00289555e-02,  8.92155540e+00],
       [ 2.82737493e+00,  4.65386710e-02,  5.91945531e+00],
       [ 2.97968528e+00,  5.59116887e-02,  9.39252729e+00],
       [ 2.99625546e+00,  5.36535158e-02,  7.63980642e+00],
       [ 3.30110957e+00,  4.72463817e-02,  8.08016791e+00],
       [ 3.42557126e+00,  5.15197758e-02,  6.19331529e+00],
       [ 2.78253640e+00,  3.49039649e-02,  6.81114797e+00],
       [ 3.55385466e+00,  3.74214658e-02,  7.66047212e+00],
       [ 3.52264228e+00,  7.03788412e-02,  8.67622907e+00],
       [ 2.84906888e+00,  6.15340605e-02,  7.99957211e+00],
       [ 2.88285328e+00,  4.35992624e-02,  8.89179211e+00],
       [ 3.28386152e+00,  5.01243049e-02,  9.01991579e+00],
       [ 2.74989284e+00,  6.18838284e-02,  7.91505903e+00],
       [ 3.47640560e+00,  4.55800761e-02,  9.87091082e+00],
       [ 2.31414672e+00,  4.36382278e-02

In [4]:
# extraindo resultado
d = np.array(dataset.loc[:,"fruta"])
d


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

Mapeando a saída:

Como nossa rede possui 3 neurônios de saída, vamos mapear qual será a saída esperada de cada neurônio para cada fruta:

Maçã = 1 0 0
Limão = 0 1 0
Melão = 0 0 1

In [5]:
# Mapeando classe para vetor
def vetorizar(d):
    saidas = []
    for fruta in d:
        if fruta == 0:
            saidas.append(np.array([1, 0, 0]))
        elif fruta == 1:
            saidas.append(np.array([0, 1, 0]))
        else:
            saidas.append(np.array([0, 0, 1]))

    return np.array(saidas)

# Mapeando vetor para classe
def classificar(vetor):
    classes = []
    for y in vetor:
        maximo = np.argmax(y)
        classes.append(maximo)
        
    return classes

In [6]:
d = vetorizar(d)

# Embaralhando os dados
shuffle = np.random.permutation(len(x))
x = x[shuffle]
d = d[shuffle]

In [7]:
# Separando dados de treino (80%) e teste (20%)
limite = int(len(x) * 0.8)

x_treino = x[0:limite]
x_teste = x[limite: ]

d_treino = d[0:limite]
d_teste = d[limite: ]

In [8]:
class MLP():
    
    def __init__(self, lr, e, neurons):
        """ Construtor """
        # Taxa de aprendizado 
        self.lr = 0.2
        
        # tolerância
        self.e = 1e-6
        
        # Quantidade de neurônios por camadas
        self.neurons = [3,3,3]
        
        
    def sigmoid(self, valor):
        '''Calcula a sigmoid de um valor'''
        return (1/(1+math.e**(-valor)))

    def sigmoid_deriv(self, valor):
        '''Calcula a derivada da função sigmoid'''
        sig = self.sigmoid(valor)
        return sig*(1 - sig)

    def activate(self, valor):
        '''Ativa as saídas do neurônio'''
        return self.sigmoid(valor)
    
    def deriv(self, valor):
        '''Calcular a derivada da função de ativação'''
        return self.sigmoid_deriv(valor)

    def evaluate(self, target, predicted):
        '''Calcula a diferença entre o valor real e o valor predito'''
        return (target - predicted)

    def predict(self, input_data, weights):
        '''Calcula a soma ponderada das entradas pelo peso'''
        return np.dot(input_data, weights).reshape(1, -1)
    
    def train(self, x, d):
        ''' 
        Definir aleatoriamente os pesos, o bias e o peso do bias
        Enquanto a diferença entre m mse_anterior e o mse_atual for maior que 'e' continua o processo 
        '''
        self.w1 = np.random.random((x.shape[1]+1,self.neurons[0]))
        self.w2 = np.random.random((self.neurons[0], self.neurons[1]))
        self.w3 = np.random.random((self.neurons[1], self.neurons[2]))
        
        epoch = 0
        last_mse = np.inf
        self.total_mse = []
        self.bias = -1
        
        while True:
            mse = 0
            for xi, target in zip(x,d):
                input_value = np.insert(xi, 0, self.bias)
                i1 = self.predict(input_value, self.w1)
                y1 = self.activate(i1)
                i2 = self.predict(y1, self.w2)
                y2 = self.activate(i2)
                i3 = self.predict(y2, self.w3)
                y3 = self.activate(i3)
                current_error = self.evaluate(target, y3)
                mse+=(current_error ** 2)

                delta3 = (target - y3) * self.deriv(i3)
                self.w3 += self.lr * np.dot(y2.T, delta3)

                delta2 = np.dot(delta3, self.w3.T) * self.deriv(i2)
                self.w2 += self.lr * np.dot(y1.T, delta2)

                delta1 = np.dot(delta2, self.w2.T) * self.deriv(i1)
                self.w1 += self.lr * np.dot(input_value.reshape(1, -1).T, delta1)

            mse = sum(mse[0]) / len(x)
            
            print(f"EPOCH: {epoch} - MSE: {mse} - |mse_ant - mse|: {abs(last_mse - mse)}")
            if abs(last_mse - mse) <= self.e:
                break
            
            self.total_mse.append(mse)
            last_mse = mse
            epoch += 1
            
            
    def test(self, x):
        ''' Dado uma lista de X, submete-os à rede'''
        results = []
        for xi in x:
            input_value = np.insert(xi, 0, self.bias)
            i1 = self.predict(input_value, self.w1)
            y1 = self.activate(i1)
            i2 = self.predict(y1, self.w2)
            y2 = self.activate(i2)
            i3 = self.predict(y2, self.w3)
            y3 = self.activate(i3)
            
            results.append(y3[0])
            
        return results

In [9]:
rede = MLP(lr = 0.1, e = 1e-4, neurons = [5, 4, 3])
rede.train(x = x_treino, d = d_treino)

EPOCH: 0 - MSE: 0.8412120948927886 - |mse_ant - mse|: inf
EPOCH: 1 - MSE: 0.6848296037939464 - |mse_ant - mse|: 0.15638249109884217
EPOCH: 2 - MSE: 0.6774248127382303 - |mse_ant - mse|: 0.00740479105571612
EPOCH: 3 - MSE: 0.677159511723948 - |mse_ant - mse|: 0.00026530101428234865
EPOCH: 4 - MSE: 0.6772280428047226 - |mse_ant - mse|: 6.853108077464753e-05
EPOCH: 5 - MSE: 0.6773016697030132 - |mse_ant - mse|: 7.362689829060187e-05
EPOCH: 6 - MSE: 0.6773712530838094 - |mse_ant - mse|: 6.958338079621207e-05
EPOCH: 7 - MSE: 0.6774390699464947 - |mse_ant - mse|: 6.78168626853104e-05
EPOCH: 8 - MSE: 0.6775058868722117 - |mse_ant - mse|: 6.681692571697262e-05
EPOCH: 9 - MSE: 0.6775718638691234 - |mse_ant - mse|: 6.597699691168746e-05
EPOCH: 10 - MSE: 0.677637021695408 - |mse_ant - mse|: 6.515782628457867e-05
EPOCH: 11 - MSE: 0.6777013569979106 - |mse_ant - mse|: 6.4335302502605e-05
EPOCH: 12 - MSE: 0.6777648646116099 - |mse_ant - mse|: 6.350761369933178e-05
EPOCH: 13 - MSE: 0.6778275406545823

EPOCH: 125 - MSE: 0.6811203475829856 - |mse_ant - mse|: 1.144144696485494e-05
EPOCH: 126 - MSE: 0.6811316453063087 - |mse_ant - mse|: 1.1297723323089315e-05
EPOCH: 127 - MSE: 0.681142801754759 - |mse_ant - mse|: 1.1156448450400092e-05
EPOCH: 128 - MSE: 0.6811538193263877 - |mse_ant - mse|: 1.1017571628690881e-05
EPOCH: 129 - MSE: 0.6811647003696868 - |mse_ant - mse|: 1.0881043299049153e-05
EPOCH: 130 - MSE: 0.6811754471847221 - |mse_ant - mse|: 1.0746815035322932e-05
EPOCH: 131 - MSE: 0.6811860620242423 - |mse_ant - mse|: 1.0614839520139974e-05
EPOCH: 132 - MSE: 0.681196547094761 - |mse_ant - mse|: 1.0485070518706507e-05
EPOCH: 133 - MSE: 0.6812069045576142 - |mse_ant - mse|: 1.0357462853272104e-05
EPOCH: 134 - MSE: 0.6812171365299947 - |mse_ant - mse|: 1.0231972380481125e-05
EPOCH: 135 - MSE: 0.6812272450859594 - |mse_ant - mse|: 1.0108555964727373e-05
EPOCH: 136 - MSE: 0.6812372322574175 - |mse_ant - mse|: 9.987171458059052e-06
EPOCH: 137 - MSE: 0.6812471000350918 - |mse_ant - mse|: 

EPOCH: 247 - MSE: 0.6818868000055289 - |mse_ant - mse|: 3.378546947541139e-06
EPOCH: 248 - MSE: 0.6818901512999153 - |mse_ant - mse|: 3.3512943863334144e-06
EPOCH: 249 - MSE: 0.6818934756254664 - |mse_ant - mse|: 3.32432555116835e-06
EPOCH: 250 - MSE: 0.6818967732617012 - |mse_ant - mse|: 3.297636234744772e-06
EPOCH: 251 - MSE: 0.6819000444839983 - |mse_ant - mse|: 3.2712222971520433e-06
EPOCH: 252 - MSE: 0.6819032895636659 - |mse_ant - mse|: 3.2450796675354e-06
EPOCH: 253 - MSE: 0.6819065087680051 - |mse_ant - mse|: 3.219204339210968e-06
EPOCH: 254 - MSE: 0.6819097023603771 - |mse_ant - mse|: 3.1935923719972337e-06
EPOCH: 255 - MSE: 0.681912870600265 - |mse_ant - mse|: 3.1682398878851714e-06
EPOCH: 256 - MSE: 0.6819160137433359 - |mse_ant - mse|: 3.1431430709272234e-06
EPOCH: 257 - MSE: 0.6819191320415017 - |mse_ant - mse|: 3.118298165794009e-06
EPOCH: 258 - MSE: 0.6819222257429796 - |mse_ant - mse|: 3.0937014778853467e-06
EPOCH: 259 - MSE: 0.6819252950923479 - |mse_ant - mse|: 3.0693

EPOCH: 370 - MSE: 0.6821537742098158 - |mse_ant - mse|: 1.2847723961373347e-06
EPOCH: 371 - MSE: 0.6821550470379053 - |mse_ant - mse|: 1.2728280894824806e-06
EPOCH: 372 - MSE: 0.6821563079288404 - |mse_ant - mse|: 1.2608909351063957e-06
EPOCH: 373 - MSE: 0.6821575568877422 - |mse_ant - mse|: 1.2489589017450342e-06
EPOCH: 374 - MSE: 0.6821587939176683 - |mse_ant - mse|: 1.2370299261599271e-06
EPOCH: 375 - MSE: 0.6821600190195808 - |mse_ant - mse|: 1.2251019124720486e-06
EPOCH: 376 - MSE: 0.6821612321923123 - |mse_ant - mse|: 1.213172731495682e-06
EPOCH: 377 - MSE: 0.6821624334325295 - |mse_ant - mse|: 1.2012402171857062e-06
EPOCH: 378 - MSE: 0.6821636227346953 - |mse_ant - mse|: 1.1893021658604397e-06
EPOCH: 379 - MSE: 0.6821648000910319 - |mse_ant - mse|: 1.1773563365347073e-06
EPOCH: 380 - MSE: 0.6821659654914755 - |mse_ant - mse|: 1.1654004435923682e-06
EPOCH: 381 - MSE: 0.682167118923638 - |mse_ant - mse|: 1.1534321625594757e-06
EPOCH: 382 - MSE: 0.6821682603727584 - |mse_ant - mse|

In [10]:
resultado = rede.test(x_teste)
resultado

[array([0.35637306, 0.29351703, 0.33499963]),
 array([0.35601064, 0.29292413, 0.33458713]),
 array([0.35601065, 0.29292414, 0.33458714]),
 array([0.35601067, 0.29292418, 0.33458716]),
 array([0.35606277, 0.2930117 , 0.33464678]),
 array([0.35603103, 0.29295867, 0.3346105 ]),
 array([0.35644327, 0.29362932, 0.33507926]),
 array([0.35657519, 0.29384021, 0.33522875]),
 array([0.3560611 , 0.29300876, 0.33464487]),
 array([0.3560604 , 0.29300778, 0.33464406]),
 array([0.35601063, 0.2929241 , 0.33458711]),
 array([0.35626396, 0.29334094, 0.33487581]),
 array([0.35601064, 0.29292413, 0.33458713]),
 array([0.35601066, 0.29292415, 0.33458715]),
 array([0.35601065, 0.29292414, 0.33458714]),
 array([0.35606452, 0.29301465, 0.33464877]),
 array([0.3564355 , 0.29361626, 0.33507052]),
 array([0.35648017, 0.29368854, 0.33512109])]

In [11]:
d_teste

array([[0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [12]:
classes_resultado = classificar(resultado)
classes_esperado = classificar(d_teste)

In [13]:
print(classes_resultado)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
print(classes_esperado)

[1, 2, 2, 2, 0, 0, 1, 1, 0, 0, 2, 1, 2, 2, 2, 0, 1, 1]


In [15]:
def acuracia(real, predito):
    acertos = 0
    for i in range(len(real)):
        if real[i] == predito[i]:
            acertos+=1
    
    return acertos/len(real)

In [16]:
acuracia(classes_esperado, classes_resultado)

0.2777777777777778